# Applied Big Data Analytics in Finance

*By Daniel Deutsch, José Lucas Barretto, Karim Jaber and Ziyad Bekkaoui*

In [1]:
import glob
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
# Matplotlib styles
plt.style.use('ggplot')
plt.rcParams.update({
    'figure.figsize': (15, 7),
    'axes.prop_cycle': plt.cycler(color=['#4C72B0', '#C44E52', '#55A868', '#8172B2', '#CCB974', '#64B5CD']),
    'axes.facecolor': '#EAEAF2'
})

# Data Import

## OHLCV

For each file provided by the professor, concatenates the given values into one single dataframe.

In [3]:
if os.path.exists("./datasets/ohlcv.csv.gz"):
    df_ohlcv = pd.read_csv("./datasets/ohlcv.csv.gz", index_col=0, )
else:
    df_ohlcv = pd.DataFrame()
    for filename in glob.glob("./datasets/ohlcv/*.xlsx"):
        try:
            df_tmp = pd.read_excel(
                filename, 
                usecols=[4, 6, 9, 10], 
                names=['', '', '', '', 'date', '', 'price', '', '', 'market_cap', 'volume'],
            )
            df_tmp['slug'] = filename
            df_ohlcv = pd.concat([df_ohlcv, df_tmp], ignore_index=True)
        except Exception:
            pass
    df_ohlcv.to_csv("./datasets/ohlcv.csv.gz")

## Crypto currencies Map

Obtains information over the cryptocurrencies map of each coin through the Coin Market Cap API.

In [4]:
# Makes the http request
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/map"
headers = {
    'Accepts': "application/json", 
    'X-CMC_PRO_API_KEY': "1a073c3f-c968-40d8-9dcf-0340a3bd5796"
}
r = requests.get(url, headers=headers)
r = r.json()

# Creates a dataframe out of the response
df_cryptomap = pd.DataFrame(r['data'])

## Sentiment

Loads the sentiment dataset provided by the professor.

In [5]:
df_sentiment = pd.read_csv("./datasets/sentiment.csv.gz", index_col=0)

# Data Processing

## OHLCV

In [6]:
# Transforms date column dtype
df_ohlcv['date'] = pd.to_datetime(df_ohlcv['date'])

# Keeps only first sample of duplicated rows
df_ohlcv.drop_duplicates(subset=['slug', 'date'], keep='first', inplace=True)

## Cryptocurrencies Map

In [7]:
# Keeps only useful columns
df_cryptomap = df_cryptomap[['symbol', 'name', 'slug']]

## Sentiment

In [8]:
# Drop incomplete rows
df_sentiment.dropna(inplace=True)

# Represents bearish sentiment with -1 (instead of 0) 
df_sentiment['sentiment'] = df_sentiment['sentiment'].replace(0, -1)

# Renames the ticker to match CoinMarketCap symbol names
df_sentiment['ticker'] = df_sentiment['ticker'].apply(lambda ticker: ticker[:-2])
df_sentiment.rename(columns={'ticker': 'symbol'}, inplace=True)

# Obtains the average sentiment of each coin per day
df_sentiment['date'] = pd.to_datetime(df_sentiment['date'])
df_sentiment = df_sentiment.set_index('date').groupby('symbol').resample('MS').mean().reset_index()

## Dataframes Merge

In [9]:
# Merges ohlcv and cryptomap dataframes
df = pd.merge(df_ohlcv, df_cryptomap, on=['slug'], how='inner')

# Merges the previously merged dataframe with the sentiment one
df = pd.merge(df_sentiment, df, on=['date', 'symbol'], how='inner')

# As some of the coins change their names/slugs over ttime but keep their symbol,
# there will be duplicated rows in the dataset.
# Keeps only first sample of duplicated rows (the most recent one)
df.drop_duplicates(subset=['symbol', 'date'], keep='first', inplace=True)

# Keeps only useful columns
df = df[['date', 'symbol', 'price', 'volume', 'market_cap', 'sentiment']]

# Transforms the dataframe into a multi-level column dataframe
df = df.set_index(['symbol', 'date']).stack().unstack('date').T

# Deletes previous dataframes from memory
del df_ohlcv, df_cryptomap, df_sentiment

df

symbol         1INCH                                                \
           sentiment     close     market_cap        volume   slug   
date                                                                 
2013-05-01       NaN       NaN            NaN           NaN    NaN   
2013-06-01       NaN       NaN            NaN           NaN    NaN   
2013-07-01       NaN       NaN            NaN           NaN    NaN   
2013-08-01       NaN       NaN            NaN           NaN    NaN   
2013-09-01       NaN       NaN            NaN           NaN    NaN   
...              ...       ...            ...           ...    ...   
2021-09-01  0.462366  3.356972   605470685.07  398167302.65  1inch   
2021-10-01 -0.036212  2.910554   524953743.28  379990830.98  1inch   
2021-11-01       0.1  4.573478   824882121.25  230861626.41  1inch   
2021-12-01      0.25  3.680926   663899538.04   492707336.3  1inch   
2022-01-01       1.0  2.530239  1069328246.05   68767576.73  1inch   

symbol                         2CRZ                                    ...  \
                     name sentiment     close  market_cap      volume  ...   
date                                                                   ...   
2013-05-01            NaN       NaN       NaN         NaN         NaN  ...   
2013-06-01            NaN       NaN       NaN         NaN         NaN  ...   
2013-07-01            NaN       NaN       NaN         NaN         NaN  ...   
2013-08-01            NaN       NaN       NaN         NaN         NaN  ...   
2013-09-01            NaN       NaN       NaN         NaN         NaN  ...   
...                   ...       ...       ...         ...         ...  ...   
2021-09-01  1inch Network       NaN       NaN         NaN         NaN  ...   
2021-10-01  1inch Network       NaN       NaN         NaN         NaN  ...   
2021-11-01  1inch Network      -1.0  0.050361  4946135.82  1012259.21  ...   
2021-12-01  1inch Network       NaN       NaN         NaN         NaN  ...   
2022-01-01  1inch Network       NaN       NaN         NaN         NaN  ...   

symbol            ZORA                                      ZRX            \
            market_cap     volume      slug      name sentiment     close   
date                                                                        
2013-05-01         NaN        NaN       NaN       NaN       NaN       NaN   
2013-06-01         NaN        NaN       NaN       NaN       NaN       NaN   
2013-07-01         NaN        NaN       NaN       NaN       NaN       NaN   
2013-08-01         NaN        NaN       NaN       NaN       NaN       NaN   
2013-09-01         NaN        NaN       NaN       NaN       NaN       NaN   
...                ...        ...       ...       ...       ...       ...   
2021-09-01  5789544.61  248523.87  zoracles  Zoracles  0.233333  1.125794   
2021-10-01  5804192.95  833101.73  zoracles  Zoracles  0.159664  0.981514   
2021-11-01         NaN        NaN       NaN       NaN  0.428571  1.234106   
2021-12-01         NaN        NaN       NaN       NaN -0.555556  1.154328   
2022-01-01         NaN        NaN       NaN       NaN      -1.0  0.866343   

symbol                                             
               market_cap        volume slug name  
date                                               
2013-05-01            NaN           NaN  NaN  NaN  
2013-06-01            NaN           NaN  NaN  NaN  
2013-07-01            NaN           NaN  NaN  NaN  
2013-08-01            NaN           NaN  NaN  NaN  
2013-09-01            NaN           NaN  NaN  NaN  
...                   ...           ...  ...  ...  
2021-09-01   951680890.73  101113528.16   0x   0x  
2021-10-01   829714255.63   96281937.45   0x   0x  
2021-11-01  1043432069.26  372953483.25   0x   0x  
2021-12-01   975979979.84   90240640.54   0x   0x  
2022-01-01   734222031.28   50251104.31   0x   0x  

[105 rows x 2778 columns]

# Data Enhancing

In [10]:
# Obtains a list with the considered symbols and factors
symbols = df.columns.levels[0]
factors = df.columns.levels[1]

for s in symbols:
    df[(s, 'market_cap')] = np.log(df[(s, 'market_cap')])
    # df[(s, 'momentum')] = 

TypeError: loop of ufunc does not support argument 0 of type float which has no callable log method

# Analysis

## Filtering

In this project we intend to only consider coins with market capitalization above $1,000,000.00

In [ ]:
# Removes unknown tickers
df_prices = df_prices[df_prices['name'].isin(ids.keys())]


In [147]:
df = pd.read_csv("./datasets/full.csv.gz", index_col=0, parse_dates=[0])
returns = df.groupby(['ticker', 'date'])[['close']].sum().stack().unstack(level=0).reset_index(level=1, drop=True)
returns = np.log(returns) - np.log(returns).shift()

c:\Users\Dell\Desktop\Code\FTD\Big Data in Finance\project\venv\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [148]:
factor = df.groupby(['ticker', 'date'])[['market_cap']].sum().stack().unstack(level=0).reset_index(level=1, drop=True)
factor = np.log(factor).shift(1)
short_mask = factor.quantile(0.1, axis=1)
long_mask = factor.quantile(0.9, axis=1)
strat = -1*factor.lt(short_mask, axis=0) + 1*factor.gt(long_mask, axis=0)

c:\Users\Dell\Desktop\Code\FTD\Big Data in Finance\project\venv\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
c:\Users\Dell\Desktop\Code\FTD\Big Data in Finance\project\venv\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
c:\Users\Dell\Desktop\Code\FTD\Big Data in Finance\project\venv\lib\site-packages\numpy\lib\function_base.py:4488: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))
c:\Users\Dell\Desktop\Code\FTD\Big Data in Finance\project\venv\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Users\Dell\Desktop\Code\FTD\Big Data in Finance\project\venv\lib\site-packages\numpy\lib\function_base.py:4488: RuntimeWarning: invalid value encountered in

In [149]:
strat_returns = (strat*returns).mean(axis=1).dropna()

In [150]:
returns

ticker,0xBTC,1ART,1INCH,1WO,2CRZ,8PAY,AAVE,ABT,ABYSS,ACE,...,ZOO,ZOON,ZORA,ZRX,ZUSD,ZYN,aEth,eRSDL,erowan,vSXP
date,,,,,,,,,,,,,,,,,,,,,
2013-04-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-28,0.212601,0.044928,-0.092231,-0.057223,-0.102878,-0.055931,-0.091442,-0.055052,-0.069836,-0.138457,...,-0.084190,-0.030902,-0.317435,-0.089680,0.006018,0.024422,-0.065989,-0.090579,-0.099131,-0.094172
2021-12-29,0.009132,0.003267,-0.052294,-0.035137,-0.028810,-0.000394,-0.069785,-0.013785,-0.090346,-0.073045,...,0.023470,-0.097930,0.517396,-0.052806,-0.000850,-0.000763,-0.041957,-0.005818,-0.010176,0.021054
2021-12-30,0.057112,0.091945,0.021075,0.019999,-0.012316,-0.067182,0.111832,0.012369,0.017480,-0.007194,...,0.061892,0.018508,-0.571277,0.047456,0.001724,-0.025223,0.019400,-0.052908,-0.023743,0.225422


In [156]:
returns

ticker,0xBTC,1ART,1INCH,1WO,2CRZ,8PAY,AAVE,ABT,ABYSS,ACE,...,ZOO,ZOON,ZORA,ZRX,ZUSD,ZYN,aEth,eRSDL,erowan,vSXP
date,,,,,,,,,,,,,,,,,,,,,
2013-04-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-28,0.212601,0.044928,-0.092231,-0.057223,-0.102878,-0.055931,-0.091442,-0.055052,-0.069836,-0.138457,...,-0.084190,-0.030902,-0.317435,-0.089680,0.006018,0.024422,-0.065989,-0.090579,-0.099131,-0.094172
2021-12-29,0.009132,0.003267,-0.052294,-0.035137,-0.028810,-0.000394,-0.069785,-0.013785,-0.090346,-0.073045,...,0.023470,-0.097930,0.517396,-0.052806,-0.000850,-0.000763,-0.041957,-0.005818,-0.010176,0.021054
2021-12-30,0.057112,0.091945,0.021075,0.019999,-0.012316,-0.067182,0.111832,0.012369,0.017480,-0.007194,...,0.061892,0.018508,-0.571277,0.047456,0.001724,-0.025223,0.019400,-0.052908,-0.023743,0.225422


In [137]:
(100*((1+strat_returns).cumprod())).tail(440)

date
2020-10-19    2.666713
2020-10-20    2.647179
2020-10-21    2.644796
2020-10-22    2.630359
2020-10-23         inf
                ...   
2021-12-28         inf
2021-12-29         inf
2021-12-30         inf
2021-12-31         inf
2022-01-01         inf
Length: 440, dtype: float64

In [ ]:
def ma_strat(long_term_window, short_term_window, returns):
    ma_long_term = prices.rolling(long_term_window).mean()
    ma_short_term = prices.rolling(short_term_window).mean()

    mask = ma_long_term.iloc[:,0].isna()
    ma_long_term = ma_long_term.loc[~mask]
    ma_short_term = ma_short_term.loc[~mask]
    returns = returns.loc[~mask]

    strat = 2*(ma_short_term > ma_long_term) - 1
    strat_returns = strat*(returns.shift(-1))
    
    return strat_returns, strat

In [ ]:
strat_returns, strat = ma_strat(50, 15, returns)

fig, ax = plt.subplots()
(1+strat_returns).cumprod().mean(axis=1).plot(ax=ax, label="strategy")
(1+returns).loc[strat_returns.index, :].cumprod().mean(axis=1).plot(ax=ax, label="market")
plt.grid(alpha=0.6)
plt.legend(loc="lower left")
plt.show()

In [ ]:
def volatility(strat_returns):
    cum_returns = strat_returns.dropna().mean(axis=1)
    annualized_vol = cum_returns.std()*np.sqrt(252)
    return annualized_vol

# RF RATE MISSING
def sharpe_ratio(strat_returns):
    cum_returns = strat_returns.dropna().mean(axis=1)
    sr = cum_returns.mean()/cum_returns.std()
    annualized_sr = sr*np.sqrt(252)
    return annualized_sr

def max_drawdown(strat_returns):
    cum_returns = strat_returns.dropna().mean(axis=1)
    max_dd = (cum_returns.iloc[-1] - cum_returns.max())/cum_returns.max()
    return max_dd


In [ ]:
sharpe_ratio(strat_returns)

In [ ]:
max_drawdown(strat_returns)

In [ ]:
tickers = ['BTC', 'ETH']
factors = ['price', 'sentiment']
columns = pd.MultiIndex.from_product([tickers, factors], names=['tickers', 'factors'])
df = pd.DataFrame(columns=columns)

In [ ]:
df['BTC', 'price']